In [1]:
!pip install sqlalchemy pandas numpy==1.24.3 psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 4.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 6.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


In [2]:
import requests
import pandas as pd
import numpy as np
import random
import os
import time

from sqlalchemy import create_engine, text
from sklearn.model_selection import train_test_split

os.environ["POSTGRES_USER"] = "myuser"
os.environ["POSTGRES_PASSWORD"] = "mypassword"
os.environ["POSTGRES_DB"] = "mydb"

In [3]:
# Extract the raw data
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
raw_df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.00632,18.00,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3
1,396.90000,4.98,24.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.02731,0.00,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8
3,396.90000,9.14,21.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.02729,0.00,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8


In [4]:
# Process the raw data
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

df = pd.DataFrame(data, 
                  columns = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 
                              'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'])
df.insert(0, "home_id", df.index + 1)
df['medv'] = target
df.head()

,home_id,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,1,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,3,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,4,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,5,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
# Define the PostgreSQL connection URL
db_url = f"postgresql://{os.environ['POSTGRES_USER']}:{os.environ['POSTGRES_PASSWORD']}@db:5432/{os.environ['POSTGRES_DB']}"
print("Database conection string", db_url)

# Create the engine
engine = create_engine(db_url)

# Insert the data into the table
df.to_sql('boston_dataset', engine, if_exists='replace', index=False)

# Verify that the data has been inserted
with engine.connect() as conn:
    result = conn.execute(text('SELECT COUNT(*) FROM boston_dataset;'))
    print(f"Number of rows in boston table: {result.fetchone()[0]}")

Database conection string postgresql://myuser:mypassword@db:5432/mydb
Number of rows in boston table: 506


In [6]:
requests.get("http://ml-api:5000/train")

<Response [200]>

In [ ]:
_, X_test, _, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.2, random_state=42)
X_test
while True:
    rand = np.random.choice(X_test.shape[0], size=1, replace=False)
    sampled_row = df.iloc[rand, :].values[0]
    payload = {k: v for k, v in zip(df.columns[:-1], sampled_row)}
    payload["home_id"] = int(payload["home_id"])
    print(payload)
    prediction = requests.get("http://ml-api:5000/predict", json=payload)
    print(prediction.json())
    print(42*"-")
    time.sleep(2)